<a href="https://colab.research.google.com/github/Jeniejean/Optimization/blob/main/AMPL_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# The only 3 lines you need to install and use AMPL with any solver on Colab
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "gurobi"],  # modules to install
    license_uuid="your-license-uuid",  # license to use
)  # instantiate AMPL object and register magics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.0 MB/s eta 0:00:00


In [3]:
%%ampl_eval
 reset;
 var pricePeak >= 0;
 var priceOff >= 0;
 var cap >= 0;
 var demandPeak = 2.253 - 0.013*pricePeak + 0.003*priceOff;
 var demandOff = 1.142 - 0.015*priceOff + 0.005*pricePeak;
 maximize profit: pricePeak*demandPeak + priceOff*demandOff- 75*cap;
 s.t. conPeak: demandPeak <= cap;
 s.t. conOff: demandOff <= cap;
let pricePeak := 80;
 let priceOff := 80;
solve;
 display pricePeak, priceOff, cap;

Error executing "solve" command:
No solver specified:  option solver is ''.
pricePeak = 80
priceOff = 80
cap = 0



In [4]:
#Modify code as homework5
%%ampl_eval
 reset;
 var pricePeak >= 0;
 var priceOff >= 0;
 var cap >= 0;
 var demandPeak = 2.253 - 0.013*pricePeak + 0.003*priceOff;
 var demandOff = 1.142 - 0.015*priceOff + 0.005*pricePeak;
 maximize profit: pricePeak*demandPeak + priceOff*demandOff- 75*cap;
 s.t. conPeak: demandPeak <= cap;
 s.t. conOff: demandOff <= cap;
s.t. capFix: cap = 0.65;

let pricePeak := 80;
 let priceOff := 80;
solve;
 display pricePeak, priceOff, cap;

Error executing "solve" command:
No solver specified:  option solver is ''.
pricePeak = 80
priceOff = 80
cap = 0.65



In [5]:
%%ampl_eval
reset;
var pricePeak >= 0;
var priceOff >= 0;
var cap >= 0;
var demandPeak = 2.253 - 0.013*pricePeak + 0.003*priceOff;
var demandOff = 1.142 - 0.015*priceOff + 0.005*pricePeak;

maximize profit: pricePeak*demandPeak + priceOff*demandOff - 75*cap;

s.t. conPeak: demandPeak <= cap;
s.t. conOff: demandOff <= cap;
s.t. priceDifference: abs(pricePeak - priceOff) <= 40;  # Additional constraint for price difference

let pricePeak := 80;
let priceOff := 80;
solve;

display pricePeak, priceOff, cap;

Error executing "solve" command:
No solver specified:  option solver is ''.
pricePeak = 80
priceOff = 80
cap = 0



In [6]:
%%ampl_eval
reset;
set ITEMS;

param weight{ITEMS};
param value{ITEMS};
param max_weight;

var x{ITEMS} binary;

maximize total_value: sum{i in ITEMS} value[i] * x[i];

subject to weight_constraint: sum{i in ITEMS} weight[i] * x[i] <= max_weight;

In [7]:
from random import randint

items = list(range(1, 10 + 1))
ampl.set["ITEMS"] = items
ampl.param["weight"] = {i: randint(10, 20) for i in items}
ampl.param["value"] = {i: randint(100, 200) for i in items}
ampl.param["max_weight"] = 100
ampl.option["solver"] = "highs"
ampl.solve()
ampl.get_data("x").to_pandas()

HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 1131
2 simplex iterations
1 branching nodes


,x
1,1
2,1
3,1
4,0
5,1
6,0
7,1
8,1
9,0
10,1


[AMPL Website](https://ampl.com) | [AMPL Colab](https://colab.ampl.com) | [Community Edition]( https://ampl.com/ce/) | [Twitter](https://twitter.com/AMPLopt) | [LinkedIn](https://www.linkedin.com/company/ampl)

[![Hits](https://h.ampl.com/https://github.com/ampl/amplcolab/blob/master/template/minimal.ipynb)](https://colab.ampl.com)

In [8]:
%%ampl_eval
reset;

set ORG;
set DEST;

param cost{i in ORG, j in DEST};
param supply{i in ORG};
param demand{j in DEST};

var x{i in ORG, j in DEST} >= 0, <= 250;

minimize totalCost: sum{i in ORG, j in DEST} cost[i,j]*x[i,j];
s.t. conSupply{i in ORG}: sum{j in DEST} x[i,j] <= supply[i];
s.t. conDemand{j in DEST}: sum{i in ORG} x[i,j] >= demand[j];

data;

set ORG := P1 P2 P3;
set DEST := R1 R2 R3 R4;

param cost : R1 R2 R3 R4 :=
P1 131	218	266	120
P2 250	116	263	278
P3 178	132	122	180;

param supply := P1 450 P2 600  P3 500;
param demand := R1 450 R2 200  R3 300 R4 300;

solve;
display x;
display totalCost;

HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 180150
6 simplex iterations
0 barrier iterations
x :=
P1 R1   200
P1 R2     0
P1 R3     0
P1 R4   250
P2 R1    50
P2 R2   200
P2 R3    50
P2 R4     0
P3 R1   200
P3 R2     0
P3 R3   250
P3 R4    50
;

totalCost = 180150



In [9]:
#สมมติถ้าเส้นทาง14พัง
%%ampl_eval
reset;

set ORG;
set DEST;

param cost{i in ORG, j in DEST};
param supply{i in ORG};
param demand{j in DEST};

#add arc constrain <250
var x{i in ORG, j in DEST} >= 0, <= 250;

minimize totalCost: sum{i in ORG, j in DEST} cost[i,j]*x[i,j];
s.t. conSupply{i in ORG}: sum{j in DEST} x[i,j] <= supply[i];
s.t. conDemand{j in DEST}: sum{i in ORG} x[i,j] >= demand[j];

#add on
s.t. blockRoute: x["P1", "R4"] = 0;

data;

set ORG := P1 P2 P3;
set DEST := R1 R2 R3 R4;

param cost : R1 R2 R3 R4 :=
P1 131	218	266	120
P2 250	116	263	278
P3 178	132	122	180;

param supply := P1 450 P2 600  P3 500;
param demand := R1 450 R2 200  R3 300 R4 300;

solve;
display x;
display totalCost;

HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 208500
4 simplex iterations
0 barrier iterations
x :=
P1 R1   250
P1 R2     0
P1 R3     0
P1 R4     0
P2 R1   200
P2 R2   200
P2 R3    50
P2 R4    50
P3 R1     0
P3 R2     0
P3 R3   250
P3 R4   250
;

totalCost = 208500



In [10]:
#สมมติถ้าเส้นทาง14พัง
%%ampl_eval
reset;

set ORG;
set DEST;

param cost{i in ORG, j in DEST};
param supply{i in ORG};
param demand{j in DEST};

var x{i in ORG, j in DEST} >= 0;

minimize totalCost: sum{i in ORG, j in DEST} cost[i,j]*x[i,j];
s.t. conSupply{i in ORG}: sum{j in DEST} x[i,j] <= supply[i];
s.t. conDemand{j in DEST}: sum{i in ORG} x[i,j] >= demand[j];

#add arc constrain <250
s.t. conArc{i in ORG, j in DEST}: x[i,j] <= 250;

#add on
s.t. blockRoute: x["P1", "R4"] = 0;

data;

set ORG := P1 P2 P3;
set DEST := R1 R2 R3 R4;

param cost : R1 R2 R3 R4 :=
P1 131	218	266	120
P2 250	116	263	278
P3 178	132	122	180;

param supply := P1 450 P2 600  P3 500;
param demand := R1 450 R2 200  R3 300 R4 300;

solve;
display x;
display totalCost;


HiGHS 1.7.1: HiGHS 1.7.1: optimal solution; objective 208500
4 simplex iterations
0 barrier iterations
x :=
P1 R1   250
P1 R2     0
P1 R3     0
P1 R4     0
P2 R1   200
P2 R2   200
P2 R3    50
P2 R4    50
P3 R1     0
P3 R2     0
P3 R3   250
P3 R4   250
;

totalCost = 208500



In [13]:
ampl.option["solver"] = "gurobi"
ampl.solve()

Gurobi 11.0.3: Gurobi 11.0.3: optimal solution; objective 1535562.5
0 simplex iterations


In [14]:
#Uncapacitated lot sizing
%%ampl_eval
reset;

param n;  # number of periods
set TIME := 1..n;
param capProd;  # Maximum production capacity
param capWH;    # Maximum warehouse capacity
param inv0;     # Initial inventory
param demand{TIME};
param prodCost{TIME};
param holdCost{t in TIME} = 0.05*prodCost[t];
param fixedCost = 15000;
param bigM = 130000;   # Large number to link production and binary variable

var x{TIME} >= 0 ;
var inv{TIME} >= 0 ;
var y{TIME} binary;             # Binary variable for fixed cost

minimize total_cost: sum{t in TIME}(prodCost[t]*x[t] + holdCost[t]*inv[t] + fixedCost*y[t]);

s.t. inv_balance{t in 2..n}: inv[t] = inv[t-1] + x[t] - demand[t];
s.t. init_inv: inv[1] = inv0 + x[1] - demand[1];

# Link production with binary variable (M*Binary)
s.t. link_prod{t in 1..n}: x[t] <= bigM * y[t];

data;

param n := 6;
param capProd := 30000;
param inv0 := 5000;
param prodCost := 1 12.50 2 12.55 3 12.70 4 12.80 5 12.85 6 12.95;
param demand := 1 10000  2 15000  3 30000   4 35000  5  25000 6 10000;

solve;
display x;
display y;
display total_cost;

Gurobi 11.0.3: Gurobi 11.0.3: optimal solution; objective 1604550
14 simplex iterations
1 branching node
absmipgap=4.65661e-10, relmipgap=2.90213e-16
x [*] :=
1  20000
2      0
3  30000
4  35000
5  35000
6      0
;

y [*] :=
1  1
2  0
3  1
4  1
5  1
6  0
;

total_cost = 1604550



AMPLException: line 32 offset 680
syntax error
context:  ampl >>> .o <<< ption["solver"] = "gurobi"